# Demonstration of CLEM-Reg running in batch mode
This notebook demostrates how to use CLEM-Reg in a headless/batch mode configuration. A standalone python script that can be launched directly via the command line, with the same functionality, is also provided, for example to be submitted to a High Performance Computing (HPC) scheduler.

## Data
### Inputs
The data we are using is associated with [this deposition in EMPIAR](https://www.ebi.ac.uk/empiar/EMPIAR-10819/), as presented in the CLEM-Reg manuscript. A sample dataset derived from this deposition is shared via zenodo [here](https://zenodo.org/records/7936982) so that it is accessible via the Sample Data menu in the napari GUI, and we use the same link to retrieve it in this notebook.

### Outputs
The notebook outputs a set of warped image files, with each warped channel saved as an individual output file, named with the convention `<CHANNEL_NAME>_warped.tif`. Output files are saved by default to WHERE_SAVED. 

## How to run this notebook
INSERT GENERIC JUPYTER INSTRUCTIONS, OR LINK TO ONLINE INFO

# Import required packages

In [44]:
import napari 
import numpy as np

import os
from skimage.io import imread, imsave
import torch

from napari.layers import Labels, Points, Image
from napari.experimental import link_layers

from napari_clemreg.clemreg.log_segmentation import log_segmentation
from napari_clemreg.clemreg.empanada_segmentation import empanada_segmentation
from napari_clemreg.clemreg.sample_data import make_sample_data
from napari_clemreg.clemreg.point_cloud_sampling import point_cloud_sampling
from napari_clemreg.clemreg.point_cloud_registration import point_cloud_registration
from napari_clemreg.clemreg.warp_image_volume import warp_image_volume

Check if there's a CUDA enabled GPU present, if not we won't run the EM segmentation step and use a pre-computed mask instead

In [45]:
#has_gpu = torch.cuda.is_available()

has_gpu = False # For testing, set to always false

if has_gpu:
    print("Found CUDA, so we'll run the full EM segmentation")
else:
    print("No CUDA found, so we'll use a pre-computed EM segmentation to save time")

No CUDA found, so we'll use a pre-computed EM segmentation to save time


## Loading the data
Here we run a function to download the `napari-clemreg` example data from zenodo. The function returns a tuple containing the data and metadata for one volume EM dataset and the corresponding fluorescence microscopy datasets to be aligned.

We use napari's `Image` and `Labels` types to allow us to correctly propagate metadata in a way consistent with the napari plugin

This cell prepares the input data as `Image` data objects. To test on different data, you can replace the file with a different path, ensuring you add the appropriate image voxel resolution via the `scale` argument to `Image`

In [46]:
em, tgn, lyso, mito, nuc = make_sample_data()  

em_scale = [0.02, 0.02, 0.02]            # Note that the scales are provided in microns, 
lm_scale = [0.13, 0.0352740, 0.0352740]  # in the order [Z, Y, X] for both EM and LM data

# Create napari "Image" objects with raw data and metadata
em_image = Image(em[0], metadata=em[1], name='EM', scale=em_scale)
fm_tgn   = Image(tgn[0], metadata=tgn[1], name='TGN', scale=lm_scale)
fm_lyso  = Image(lyso[0], metadata=lyso[1], name='Lysosomes', scale=lm_scale)
fm_mito  = Image(mito[0], metadata=mito[1], name='Mitochondria', scale=lm_scale)
fm_nuc   = Image(nuc[0], metadata=nuc[1], name='Nucleus', scale=lm_scale)

# Link the layers so that the transform is applied across all FM channels
_ = link_layers([fm_tgn, fm_lyso, fm_mito, fm_nuc])

Loading EM...
Loading FM...


If we have a CUDA enabled GPU, we can run the EM segmentation, otherwise retrieve a pre-computed segmentation from the GitHub repository. Any appropriate segmentation function can be substituted here.

## Segmenting the data
In case the user doesn't have a GPU, we've supplied the EM mask as created with MitoNet default settings in the napari plugin for demonstration purposes

In [47]:
if has_gpu:
    # Perform MitoNet segmentation
    em_mask_data = empanada_segmentation(input=em_image.data, axis_prediction=False)
else:
    em_mask_file = "data/em_mask.tif"
    em_mask_data = imread(em_mask_file)

em_mask=Labels(np.uint16(em_mask_data), scale=em_scale)

In [48]:
fm_mito_mask_data = log_segmentation(fm_mito)
# fm_mito_mask_metadata = {'name': 'mito_segmentation',
#                          'metadata': {'XResolution': fm_mito.metadata['metadata']['XResolution'],
#                          'YResolution': fm_mito.metadata['metadata']['YResolution']}}

fm_mito_mask = Labels(fm_mito_mask_data, scale=lm_scale)

Segmenting Mitochondria with sigma=3 and threshold=1.2...


Segmenting image..: 100%|████████████████████████████████████████████| 21/21 [00:00<00:00, 51.86it/s]


Finished segmenting after 3.1820452213287354s!


## Point cloud sampling

In [49]:
em_points = point_cloud_sampling(em_mask, voxel_size=15, every_k_points=100//3, sigma=1.0)
fm_points = point_cloud_sampling(fm_mito_mask, voxel_size=15, every_k_points=100//3, sigma=1.0)

Sampling point cloud from Labels with voxel_size=15 and sampling_frequency=0.030303030303030304...
Finished point cloud sampling after 15.0485360622406s!
Sampling point cloud from fm_mito_mask_data with voxel_size=15 and sampling_frequency=0.030303030303030304...
Finished point cloud sampling after 1.9133620262145996s!


## Calculating the transformation

In [50]:
moving_points_data, fixed_points_data, tf_moving_points_data, tf_data = point_cloud_registration(moving=fm_points, fixed=em_points)

Registering point clouds...: 100%|███████████████████████████████████| 50/50 [00:31<00:00,  1.60it/s]

time:  31.185267210006714
result:  [[ 0.99998982 -0.0038598  -0.0023362 ]
 [ 0.00378351  0.99948642 -0.03182102]
 [ 0.00245782  0.03181185  0.99949085]] 1.706058339485949 [  24.89766269  -61.65528903 -324.51593298]


In [51]:
# Expected result:  [[ 0.99998008 -0.00482832 -0.00406519]
#  [ 0.00467302  0.99929012 -0.03738212]
#  [ 0.00424279  0.03736237  0.99929278]] 0.9758678903218879 [   4.00501837  -62.06598641 -339.93251389]


In [52]:
moving_points_kwargs = dict(
    name='Moving_point_cloud',
    face_color='red',
    edge_color='black',
    size=5,
    metadata={'pxlsz': fm_mito_mask_metadata['metadata']['XResolution']}
)

fixed_points_kwargs = dict(
    name='Fixed_point_cloud',
    face_color='blue',
    edge_color='black',
    size=5,
    metadata={'pxlsz': em_image.metadata['metadata']['XResolution'], 'output_shape': em_mask.data.shape}
)

moving_points = Points(moving_points_data, **moving_points_kwargs)
fixed_points = Points(fixed_points_data, **fixed_points_kwargs)
transformed_points = Points(moving_points_data, **moving_points_kwargs) # Duplicate of the original data...
transformed_points.affine.affine_matrix = tf_data['affine']             # ...which we apply the transform to

## Applying the transformation

In [53]:
warped_images = warp_image_volume(moving_image=fm_mito,
                                  output_shape=em_image.data.shape,
                                  transform_type='Rigid CPD',
                                  moving_points=moving_points,
                                  transformed_points=transformed_points)

In [54]:
save_path_root = '/Users/jonesma/Downloads'   # CHANGE THIS TO A LOCATION OF YOUR CHOICE
for warped_image in warped_images:
    save_path = os.path.join(save_path_root, warped_image[1]['name']+'.tif')
    print(f'Saving {save_path}')
    imsave(save_path, warped_image[0], check_contrast=False)

Saving /Users/jonesma/Downloads/Lysosomes_warped.tif
Saving /Users/jonesma/Downloads/Mitochondria_warped.tif
Saving /Users/jonesma/Downloads/Nucleus_warped.tif
Saving /Users/jonesma/Downloads/TGN_warped.tif
